In [1]:
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, DDIMScheduler
from peft import PeftModel
import torch

/home/j/.local/share/virtualenvs/cv-week-2024-oEk73mMq/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "sd-legacy/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cpu")

# Проверяем, что все компоненты модели в FP16 и на cuda
assert pipe.unet.dtype == torch.float16# and pipe.unet.device.type =='cuda'
assert pipe.vae.dtype == torch.float16# and pipe.vae.device.type == 'cuda'
assert pipe.text_encoder.dtype == torch.float16# and pipe.text_encoder.device.type == 'cuda'

# Заменяем дефолтный сэмплер на DDIM
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")
#pipe.scheduler.timesteps = pipe.scheduler.timesteps.cuda()
#pipe.scheduler.alphas_cumprod = pipe.scheduler.alphas_cumprod.cuda()

unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet")
unet.to('cpu')

assert unet.dtype == torch.float32
assert unet.training

Loading pipeline components...: 100%|██████████| 7/7 [00:37<00:00,  5.34s/it]
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fai

AssertionError: 

In [ ]:
cm_unet = PeftModel.from_pretrained(
    unet,
    "jmpleo/cv-week-2024",
    subfolder='multi-cd',
    adapter_name="multi-cd",
)

In [ ]:
pipe.unet = cm_unet.eval().to(torch.float16)
assert cm_unet.active_adapter == 'multi-cd'

generator = torch.Generator(device="cpu").manual_seed(1)

images = pipe(
    prompt="A sad puppy with large eyes",
    num_inference_steps=4,
    generator=generator,
    num_images_per_prompt=4,
    guidance_scale=1
).images

images[0]